In [1]:
getwd()
wd <- getwd()
output_path <- paste(wd,"/rxn_3o",sep="")
setwd(output_path)
getwd()

[1] "/Users/graysonboyer/Charge Balance and Aqueous Speciation"

[1] "/Users/graysonboyer/Charge Balance and Aqueous Speciation/rxn_3o"

In [2]:
#there should be a .3o, c.3o and pH.3o for each sample, thus div by 3
numberofsamples <- length(list.files(pattern = "\\.3o$"))/3

for(i in 1:numberofsamples) {
    
  setwd(output_path)
  
    FileRead <- toString(readLines(paste(c(toString(i),".3o"),collapse="")))

  filenumber <- i
   
  match <- regexec("Sample:\\s*\\w+", FileRead)
  if (match[[1]][1] != -1){
    Sample_Name <- regmatches(FileRead, match)
    Sample_Name <- tail(strsplit(Sample_Name[[1]],split=" ")[[1]],1)
  }else{
    Sample_Name <- -999
  }

  match <- regexec("(\\S+)\\s*per cent of the total charge", FileRead)
  if (match[[1]][1] != -1){
    Percent_Imbalance <- regmatches(FileRead, match)
    Percent_Imbalance <- head(strsplit(Percent_Imbalance[[1]],split=" ")[[1]],1)
  }else{
    Percent_Imbalance <- -999
  }

  match <- regexec("NBS pH scale                \\S+     \\S+", FileRead)
  if (match[[1]][1] != -1){
    Eh_calc <- regmatches(FileRead, match)
    Eh_calc <- tail(strsplit(Eh_calc[[1]],split=" ")[[1]],1)
  }else{
    Eh_calc <- -999
  }

  match <- regexec("Ionic strength [[:punct:]]I[[:punct:]]= \\S+", FileRead)
  if (match[[1]][1] != -1){
    IS_calc <- regmatches(FileRead, match)
    IS_calc <- tail(strsplit(IS_calc[[1]],split=" ")[[1]],1)
  }else{
    IS_calc <- -999
  }

  FileRead2 <- toString(readLines(paste(c(toString(i),"ph.3o"),collapse="")))
  match <- regexec("Final[[:blank:]]+[[:alnum:]]+[[:punct:]][[:alnum:]]+", FileRead2)
  if (match[[1]][1] != -1){
    pH_balanced_calc <- regmatches(FileRead2, match)
    pH_balanced_calc <- tail(strsplit(pH_balanced_calc[[1]],split=" ")[[1]],1)
    pH_balanced_calc <- substr(pH_balanced_calc, 1, nchar(pH_balanced_calc)-1)
  }else{
    pH_balanced_calc <- -999
  }

  FileRead3 <- toString(readLines(paste(c(toString(i),"c.3o"),collapse="")))
  match <- regexec("Final[[:blank:]]+[[:alnum:]]+[[:punct:]][[:alnum:]]+", FileRead3)
  match2 <- regexec("[[:alnum:]]+[[:punct:]][[:alnum:]]+[[:blank:]]+mg/L HCO3-", FileRead3)

  if (match[[1]][1] != -1){
    carb_balanced_calc <- regmatches(FileRead3, match)
    carb_balanced_calc <- tail(strsplit(carb_balanced_calc[[1]],split=" ")[[1]],1)
    carb_balanced_calc <- substr(carb_balanced_calc, 1, nchar(carb_balanced_calc)-1)
  }else if(match2[[1]][1] != -1){
    carb_balanced_calc <- regmatches(FileRead3, match2)
    carb_balanced_calc <- head(strsplit(carb_balanced_calc[[1]],split=" ")[[1]],1)
    carb_balanced_calc <- substr(carb_balanced_calc, 1, nchar(carb_balanced_calc)-1)
  }else{
    carb_balanced_calc <- -999
  }

  setwd(wd)

  g <- as.matrix(read.csv("MasterInput_cleaned.csv", header=FALSE, colClasses = "character"))
  colnames(g) <- g[1,]
  colnames(g)[9] <- "HCO3._molality"
  colnames(g)[10] <- "CaCO3alkalinity"
  geochem <- g[i+2,seq(2,length(g[i,]),1)]

  ProtonErrorPercent <- 100*(((10^-(as.numeric(pH_balanced_calc)))-(10^-(as.numeric(g[i+2,"H+"]))))/(10^-(as.numeric(g[i+2,"H+"]))))

  carbinputDIC <- as.numeric(g[i+2,"HCO3._molality"])*1000*61.0168
  carbinputALK <- as.numeric(g[i+2,"CaCO3alkalinity"])*1.22 #convert CaCO3 mg/L alkalinity to HCO3- mg/L alkalinity

  if(carbinputDIC > 0){
    carbinput <- carbinputDIC
    carbmessage <- "DIC"
  }

  if(carbinputDIC == 0 && carbinputALK > 0){
    carbinput <- carbinputALK
    carbmessage <- "CaCO3"
  }

  if(carbinputDIC == 0 && carbinputALK == 0){
    carbinput <- 0
    carbmessage <- "No DIC or CaCO3"
  }

  DIC_Error <- as.numeric(carb_balanced_calc)-carbinput

  number_of_columns <- 12+length(g[i,])-1 # 1) file 2) sample 3) pH 4)charge-balanced pH 5) proton error percent 6) %imbalance 7) Eh 8) IS 9+) geochem's column length -1 (redundant geochem sample column removed)

  headers1 <- data.frame(matrix(c("","","","","HCO3- input","Charge-balanced HCO3-","Method","Charge Imbalance", "[H+] discrepancy", "HCO3- discrepancy", "Eh", "IS", g[1,seq(2,length(g[1,]),1)]),nrow=1,ncol=number_of_columns))
  headers2 <- data.frame(matrix(c("File","Sample","pH","Charge-balanced pH","mg/L","mg/L","Type","%", "%","mg/L", "volts", "mol/kg", g[2,seq(2,length(g[1,]),1)]),nrow=1,ncol=number_of_columns))
  sample_entry <- data.frame(matrix(c(filenumber, Sample_Name, g[i+2,"H+"], pH_balanced_calc, carbinput, carb_balanced_calc, carbmessage, Percent_Imbalance, ProtonErrorPercent, DIC_Error, Eh_calc, IS_calc, geochem),nrow=1,ncol=number_of_columns))

  setwd(wd)
    if(i==1){
      write.table(headers1, file="report.csv", sep = ",", append=FALSE, row.names = FALSE, col.names=FALSE)
      write.table(headers2, file="report.csv", sep = ",", append=TRUE, row.names = FALSE, col.names=FALSE)
      write.table(sample_entry, file="report.csv", sep = ",", append=TRUE, row.names = FALSE, col.names=FALSE)
    }else{
      write.table(sample_entry, file="report.csv", sep = ",", append=TRUE, row.names = FALSE, col.names=FALSE)
    }
} #end loop